**Table of contents**<a id='toc0_'></a>    
- [Feature selection](#toc1_1_)    
    - [Principle Component Analysis](#toc1_1_1_)    
    - [Feature importance from tree-based models (e.g., Random Forest)](#toc1_1_2_)    
- [model](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import sys
import os

import warnings
warnings.filterwarnings("ignore")
sys.path.append('/Users/jianbinlin/Documents/Master-Data-Spreadsheet/script/')

import utility
from utility import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(HTML("<style>.container { width:50% !important; }</style>"))
#display(HTML('<style>.output { max-width:800px !important; }</style>')) # control output width
display(HTML("<style>.container { width:130% !important; }</style>"))

pd.set_option('display.max_columns', None)   # Show number of columns, None means show all columns
pd.set_option('display.max_rows', 50)        # show top and bottom 15, middle with ......
pd.set_option('display.width', None)         # Set the width of the display
pd.set_option('display.max_colwidth', None)  # Show full content of each column

24/12/10 12:50:19 WARN Utils: Your hostname, Jians-iMac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.23 instead (on interface en1)
24/12/10 12:50:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 12:50:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
term_deposit_subscription = pd.read_csv('https://raw.githubusercontent.com/KevinJianLin/term_deposit_subscription/refs/heads/main/data/train.csv')
data_set = term_deposit_subscription.copy()
data_set.head(5)
data_profile = data_profiling(data_set)
#data_profile()
# data_profile.float_column
# data_profile.int_column
# data_profile.rest_columns
# data_profile.cat_col
target_variable = 'subscribed' 

# Prepare the feature matrix and target
X = data_set[data_profile.int_column]
y = data_set[target_variable]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,26110,56,admin.,married,unknown,no,1933,no,no,telephone,19,nov,44,2,-1,0,unknown,no
1,40576,31,unknown,married,secondary,no,3,no,no,cellular,20,jul,91,2,-1,0,unknown,no
2,15320,27,services,married,secondary,no,891,yes,no,cellular,18,jul,240,1,-1,0,unknown,no
3,43962,57,management,divorced,tertiary,no,3287,no,no,cellular,22,jun,867,1,84,3,success,yes
4,29842,31,technician,married,secondary,no,119,yes,no,cellular,4,feb,380,1,-1,0,unknown,no


In [ ]:
data_preprocessor_1 = ColumnTransformer(transformers = [
                                   #         ('text', TfidfVectorizer(max_features=500), text_col),
                                            ('float_col',StandardScaler(),data_profile.int_column),
                                        #    ('cat_col', OneHotEncoder(), cat_col),
                                            ]) 

label_encoder = LabelEncoder()
data_set[target_variable] = label_encoder.fit_transform(data_set[target_variable])


## <a id='toc1_1_'></a>[Feature selection](#toc0_)

### <a id='toc1_1_1_'></a>[Principle Component Analysis](#toc0_)

### <a id='toc1_1_2_'></a>[Feature importance from tree-based models (e.g., Random Forest)](#toc0_)
Permutation importance
SHAP (SHapley Additive exPlanations) values
Recursive feature elimination
Lasso regularization

### 

# <a id='toc2_'></a>[model](#toc0_)

In [12]:
best_model ={}

In [ ]:
# Loop through models and apply GridSearchCV
for model_name, model_infor in model_parameters_classification.items():


    # Create a pipeline for each model
    pipeline = Pipeline([('preprocessor',data_preprocessor_1),
                         ('classifier',model_infor['model'])])
    
    # Initialize GridSearchCV with model and parameter grid
    grid_search = GridSearchCV(estimator=pipeline,
                                param_grid = model_infor['params'],
                                cv = 5, # 5 fold cv
                                scoring = 'accuracy', # f1 etc
                                verbose =0, # not to display output
                                n_jobs = -1 # use all of cores
                                )
    grid_search.fit(X_train, y_train)

    best_model[model_name] = grid_search.best_estimator_
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")

    y_pred = grid_search.predict(X_test)
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(y_test, y_pred, target_names = label_encoder.classes_))

    print(model_infor['model'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('float_col',
                                                                         StandardScaler(),
                                                                         ['ID',
                                                                          'age',
                                                                          'balance',
                                                                          'day',
                                                                          'duration',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous'])])),
                                       ('classifier',
                                        DecisionTreeClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10]},
             scoring='accuracy')

Best Parameters for decisiontree_classifier: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10}
Classification Report for decisiontree_classifier:

              precision    recall  f1-score   support

          no       0.94      0.95      0.94      5599
         yes       0.57      0.53      0.55       731

    accuracy                           0.90      6330
   macro avg       0.76      0.74      0.75      6330
weighted avg       0.90      0.90      0.90      6330

DecisionTreeClassifier(random_state=42)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('float_col',
                                                                         StandardScaler(),
                                                                         ['ID',
                                                                          'age',
                                                                          'balance',
                                                                          'day',
                                                                          'duration',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__bootstrap': [True, False],
                         'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': [None, 20],
                         'classifier__max_features': ['sqrt', 'log2', None],
                         'classifier__max_samples': [0.0, 1.0],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='accuracy')

Best Parameters for randomforest_classifier: {'classifier__bootstrap': True, 'classifier__criterion': 'entropy', 'classifier__max_depth': 20, 'classifier__max_features': 'log2', 'classifier__max_samples': 1.0, 'classifier__n_estimators': 100}
Classification Report for randomforest_classifier:

              precision    recall  f1-score   support

          no       0.94      0.96      0.95      5599
         yes       0.61      0.53      0.57       731

    accuracy                           0.91      6330
   macro avg       0.78      0.74      0.76      6330
weighted avg       0.90      0.91      0.90      6330

RandomForestClassifier(random_state=42)
